In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
import sklearn.metrics as metrics

In [3]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'sad', 'angry']

In [4]:

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("./data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=42)

In [5]:
x_train, x_test, y_train, y_test=load_data(test_size=0.2)

In [6]:
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


In [7]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


## 1. Multilayer perceptron

In [8]:
def MLP(x_train, y_train):
    # define the multinomial logistic regression model
    model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
    #DataFlair - Train the model
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

    return n_scores

In [9]:
scores = MLP(x_train, y_train)

In [10]:
scores

array([0.67741935, 0.67741935, 0.51612903, 0.56451613, 0.63934426,
       0.55737705, 0.68852459, 0.67213115, 0.62295082, 0.54098361,
       0.64516129, 0.62903226, 0.69354839, 0.56451613, 0.67213115,
       0.68852459, 0.54098361, 0.67213115, 0.70491803, 0.52459016,
       0.46774194, 0.61290323, 0.56451613, 0.51612903, 0.63934426,
       0.6557377 , 0.54098361, 0.70491803, 0.72131148, 0.80327869])

In [11]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.624 (0.077)


In [12]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [13]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [14]:
y_pred=model.predict(x_test)

In [15]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.13%


In [16]:
f1=f1_score(y_true=y_test, y_pred=y_pred, average="macro")
f1

0.6911935466023946

In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       angry       0.84      0.68      0.75        38
        calm       0.62      0.95      0.75        40
       happy       0.64      0.77      0.70        35
         sad       0.85      0.41      0.56        41

    accuracy                           0.70       154
   macro avg       0.74      0.71      0.69       154
weighted avg       0.74      0.70      0.69       154



## 2. Logistic regression

In [18]:
def logistic_regression(x_train, y_train):
    # define the multinomial logistic regression model
    model = LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=500,  penalty='l2')
    #DataFlair - Train the model
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

    return n_scores

In [19]:
scores = logistic_regression(x_train, y_train)

In [20]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.635 (0.053)


In [21]:
model = LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=500,  penalty='l2')

In [22]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=500, multi_class='multinomial', solver='newton-cg')

In [23]:
y_pred=model.predict(x_test)

In [24]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 61.69%


In [25]:
f1=f1_score(y_true=y_test, y_pred=y_pred, average="macro")
f1

0.616922118082222

In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       angry       0.74      0.68      0.71        38
        calm       0.65      0.75      0.70        40
       happy       0.61      0.54      0.58        35
         sad       0.48      0.49      0.48        41

    accuracy                           0.62       154
   macro avg       0.62      0.62      0.62       154
weighted avg       0.62      0.62      0.62       154



## 3. SVM

In [27]:
def SVM(x_train, y_train):
    # define the multinomial logistic regression model
    model = svm.SVC(kernel='poly', gamma=0.5, C=0.1)
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

    return n_scores

In [28]:
scores = SVM(x_train, y_train)

In [29]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.677 (0.054)


In [30]:
scores

array([0.67741935, 0.64516129, 0.59677419, 0.58064516, 0.6557377 ,
       0.7704918 , 0.70491803, 0.72131148, 0.63934426, 0.68852459,
       0.75806452, 0.66129032, 0.64516129, 0.67741935, 0.6557377 ,
       0.70491803, 0.80327869, 0.57377049, 0.62295082, 0.6557377 ,
       0.67741935, 0.67741935, 0.64516129, 0.64516129, 0.68852459,
       0.6557377 , 0.63934426, 0.73770492, 0.72131148, 0.7704918 ])

In [31]:
model = svm.SVC(kernel='poly', gamma=0.5, C=0.1)

In [32]:
model.fit(x_train,y_train)

SVC(C=0.1, gamma=0.5, kernel='poly')

In [33]:
y_pred=model.predict(x_test)

In [34]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 69.48%


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       angry       0.77      0.79      0.78        38
        calm       0.66      0.78      0.71        40
       happy       0.69      0.63      0.66        35
         sad       0.67      0.59      0.62        41

    accuracy                           0.69       154
   macro avg       0.70      0.69      0.69       154
weighted avg       0.69      0.69      0.69       154



## 2. Random forest

In [36]:
def random_forest(x_train, y_train):
    # define the multinomial logistic regression model
    model = RandomForestClassifier(n_estimators=120)
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

    return n_scores

In [37]:
scores = random_forest(x_train, y_train)

In [38]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.709 (0.063)


In [39]:
model = RandomForestClassifier(n_estimators=120, random_state=7)

In [40]:
model.fit(x_train,y_train)

RandomForestClassifier(n_estimators=120, random_state=7)

In [41]:
y_pred=model.predict(x_test)

In [42]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.78%


In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       angry       0.88      0.74      0.80        38
        calm       0.66      0.93      0.77        40
       happy       0.59      0.69      0.63        35
         sad       0.80      0.49      0.61        41

    accuracy                           0.71       154
   macro avg       0.73      0.71      0.70       154
weighted avg       0.73      0.71      0.70       154



In [ ]:
probs = model.predict_proba(x_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()